## End of Preprocessing Script, Starting FCM

In [ ]:
from sklearn.metrics import confusion_matrix
from scipy.optimize import linear_sum_assignment

In [ ]:


fcm_results = {}   # store FCM outputs for every patient

for pid in voxel_data:
    print(f"Running FULL FCM for {pid}")

    # shape: (num_voxels, 3)
    X_pca = voxel_data[pid]["X_pca"].T  # features x samples

    # Run FCM
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
        X_pca,
        c=3,            # 3 clusters for tumor structure
        m=2.0,
        error=1e-5,
        maxiter=300,
        init=None
    )

    labels = np.argmax(u, axis=0)  # hard cluster labels

    fcm_results[pid] = {
        "centers": cntr,
        "membership": u,
        "labels": labels,
        "fpc": fpc,
        "objective": jm,
    }

    print(f"{pid}: FCM complete. FPC={fpc:.4f}, voxels={labels.shape[0]}")

# Hungarian Alignment

In [ ]:

#Copied from Kaggle Library
aligned_labels = {}

for pid in voxel_data:
    print(f"\nAligning clusters for {pid}")

    # ROI used for FCM AND GT extraction
    roi = voxel_data[pid]["roi"]

    # FCM cluster assignments (already ROI-flattened)
    labels = fcm_results[pid]["labels"]

    # Ground truth segmentation (cropped)
    seg_cropped = cropped_patients[pid]["SEG"]

    # Ground truth restricted to ROI
    true_seg_raw = seg_cropped[roi]

    # Remap BraTS labels into 0,1,2
    seg_map = {1:0, 2:1, 4:2}
    true_seg = np.array([seg_map.get(int(v), -1) for v in true_seg_raw])

    # Keep only GT where mapping is valid
    valid_idx = true_seg >= 0
    true_valid = true_seg[valid_idx]
    labels_valid = labels[valid_idx]

    # Confusion matrix
    cm = confusion_matrix(true_valid, labels_valid, labels=[0,1,2,4])

    # Hungarian alignment
    row_ind, col_ind = linear_sum_assignment(-cm)
    mapping = dict(zip(col_ind, row_ind))
    print(f"{pid}: Mapping used: {mapping}")

    # Apply mapping to ALL labels
    aligned = np.array([mapping.get(int(l), -1) for l in labels])

    aligned_labels[pid] = aligned

# Get Dice Score

In [ ]:

def dice(pred, true):
    pred = pred.astype(bool)
    true = true.astype(bool)

    if pred.sum() + true.sum() == 0:
        return 1.0
    if pred.sum() == 0 or true.sum() == 0:
        return 0.0

    return 2 * np.sum(pred & true) / (pred.sum() + true.sum())


dice_scores = {}

for pid in voxel_data:
    print(f"\nComputing {pid} Dice")

    roi = roi_patients[pid]["roi"]
    seg_cropped = cropped_patients[pid]["SEG"]

    true_seg_raw = seg_cropped[roi]


    seg_map = {1:0, 2:1, 4:2}
    true_seg = np.array([seg_map.get(int(v), -1) for v in true_seg_raw])

    # Predictions restricted to ROI (already in flattened ROI order)
    pred_seg = aligned_labels[pid]

    if len(pred_seg) != len(true_seg):
        print(f"Shape mismatch: pred={len(pred_seg)}, true={len(true_seg)}")
        continue


    true_WT = true_seg >= 0
    pred_WT = pred_seg >= 0


    true_TC = np.isin(true_seg, [0,2])
    pred_TC = np.isin(pred_seg, [0,2])


    true_ET = (true_seg == 2)
    pred_ET = (pred_seg == 2)

    WT = dice(pred_WT, true_WT)
    TC = dice(pred_TC, true_TC)
    ET = dice(pred_ET, true_ET)

    dice_scores[pid] = (WT, TC, ET)

    print(f"{pid}: WT={WT:.4f}, TC={TC:.4f}, ET={ET:.4f}")

In [ ]:
import random

def visualize_fcm_result(pid, slice_axis='z'):

    mods  = cropped_patients[pid]["masked_modalities"]
    flair = mods["FLAIR"]
    seg   = cropped_patients[pid]["SEG"]
    roi   = voxel_data[pid]["roi"]
    pred  = aligned_labels[pid]

    # Convert ROI + predictions back into arrays
    pred_full = np.zeros_like(seg) - 1
    pred_full[roi] = pred

    # Choose slice
    if slice_axis == 'z':
        slice_idx = flair.shape[2] // 2
        flair_slice = flair[:,:,slice_idx]
        seg_slice   = seg[:,:,slice_idx]
        pred_slice  = pred_full[:,:,slice_idx]
    elif slice_axis == 'y':
        slice_idx = flair.shape[1] // 2
        flair_slice = flair[:,slice_idx,:]
        seg_slice   = seg[:,slice_idx,:]
        pred_slice  = pred_full[:,slice_idx,:]
    else:
        slice_idx = flair.shape[0] // 2
        flair_slice = flair[slice_idx,:,:]
        seg_slice   = seg[slice_idx,:,:]
        pred_slice  = pred_full[slice_idx,:,:]

    # Images
    plt.figure(figsize=(14,4))

    plt.subplot(1,3,1)
    plt.title(f"{pid} — FLAIR")
    plt.imshow(flair_slice.T, cmap='gray', origin='lower')
    plt.axis("off")

    plt.subplot(1,3,2)
    plt.title("Ground Truth (SEG)")
    plt.imshow(flair_slice.T, cmap='gray', alpha=0.6, origin='lower')
    plt.imshow(seg_slice.T, cmap='jet', alpha=0.4, origin='lower')
    plt.axis("off")

    plt.subplot(1,3,3)
    plt.title("FCM Prediction (aligned)")
    plt.imshow(flair_slice.T, cmap='gray', alpha=0.6, origin='lower')
    plt.imshow(pred_slice.T, cmap='jet', alpha=0.4, origin='lower')
    plt.axis("off")

    plt.tight_layout()
    plt.show()

#Run visualization for a random patient
random_pid = random.choice(list(voxel_data.keys()))
print("Visualizing:", random_pid)
visualize_fcm_result(random_pid)